In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES']=''

import sys;
sys.path.append('..')

In [27]:
import json
wiki = json.load(open('/home/kuratov/data/ThePile/Wikipedia/wikipedia-en-0.json', 'r'))

In [42]:
{
    'title': '',
    'text': '',
}

{'title': '', 'text': ''}

In [ ]:
# articles to skip:
# List of ...
# {title} may refer to: (disambiguation pages)
# wiki key words to clean
# \nCategory:
# \n\nSee also\n\n
# \n\nReferences\n\n

In [2]:
import re
category_regexp = re.compile(r'\ACategory:[\S ]+\Z')
keywords_regexp = re.compile(r'\n(References|See also|External Links)\s?\n')
title_regexp = re.compile(r'\A[\S ]+?\\n\\n')
list_of_regexp = re.compile(r'\AList of')

def clean_article(text):
    lines = []
    title = ''
    for i, line in enumerate(text.split('\n')):
        if i == 0:
            title = line
            continue
        if category_regexp.match(line):
            continue
        lines += [line]
    text = '\n'.join(lines)
    m = re.search(keywords_regexp, text)
    if m:
        text = text[:m.span()[0]]
    return title, text.strip()

In [3]:
# preprocess wiki
from tqdm import tqdm as tqdm
import json

path = '/home/kuratov/data/ThePile/Wikipedia/wikipedia-en-{}.json'

total_articles = 0
filtered_articles = 0

articles = []

for i in tqdm(range(0, 10)):
    wiki = json.load(open(path.format(i), 'r'))
    for text in wiki:
        total_articles +=1
        title, text = clean_article(text)
        if re.search(list_of_regexp, title):
            filtered_articles += 1
            continue
        if f'{title} may refer to:' in text:
            filtered_articles += 1
            continue
        articles += [{'title': title, 'text': text}]
print(f'{filtered_articles}/{total_articles}')

100%|██████████| 10/10 [05:19<00:00, 31.99s/it]


In [6]:
print(f'{filtered_articles}/{len(articles)}/{total_articles}')

279906/5753245/6033151


In [18]:
# save shards

from pathlib import Path

shards_path = '/home/kuratov/data/ThePile/Wikipedia/preprocessed_shards'

if not Path(shards_path).exists():
    Path(shards_path).mkdir(parents=True)

import random, math
random.shuffle(articles)
n_shards = 80  # (we possibly have to take n_gpus shards for validation, so we need more shards)

shard_size = math.ceil(len(articles) / n_shards)

for i, start in enumerate(tqdm(range(0, len(articles), shard_size))):
    with (Path(shards_path) / f'shard_{i:02d}.jsonl').open('w', encoding='utf8') as fout:
        print(start, start + shard_size)
        for j in range(start, min(len(articles), start+shard_size)):
            fout.write(json.dumps(articles[j])+'\n')

100%|██████████| 80/80 [01:49<00:00,  1.36s/it]


0 71916
71916 143832
143832 215748
215748 287664
287664 359580
359580 431496
431496 503412
503412 575328
575328 647244
647244 719160
719160 791076
791076 862992
862992 934908
934908 1006824
1006824 1078740
1078740 1150656
1150656 1222572
1222572 1294488
1294488 1366404
1366404 1438320
1438320 1510236
1510236 1582152
1582152 1654068
1654068 1725984
1725984 1797900
1797900 1869816
1869816 1941732
1941732 2013648
2013648 2085564
2085564 2157480
2157480 2229396
2229396 2301312
2301312 2373228
2373228 2445144
2445144 2517060
2517060 2588976
2588976 2660892
2660892 2732808
2732808 2804724
2804724 2876640
2876640 2948556
2948556 3020472
3020472 3092388
3092388 3164304
3164304 3236220
3236220 3308136
3308136 3380052
3380052 3451968
3451968 3523884
3523884 3595800
3595800 3667716
3667716 3739632
3739632 3811548
3811548 3883464
3883464 3955380
3955380 4027296
4027296 4099212
4099212 4171128
4171128 4243044
4243044 4314960
4314960 4386876
4386876 4458792
4458792 4530708
4530708 4602624
4602624 46